In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import numpy as np
import json

# Load data from data.json
with open('../Data/data.json', 'r') as fp:
    data = json.load(fp)

# Extract MFCCs and labels
mfccs = data['mfcc']
labels = data['genre_num']

# Validate and align data
print(f"Original MFCCs length: {len(mfccs)}, Labels length: {len(labels)}")

# Ensure MFCCs and labels have the same number of samples
min_length = min(len(mfccs), len(labels))
mfccs = mfccs[:min_length]
labels = labels[:min_length]
print(f"Aligned MFCCs length: {len(mfccs)}, Labels length: {len(labels)}")

# Convert to NumPy arrays
X = np.array(mfccs)
y = np.array(labels)

# Verify shapes
print(f"X shape: {X.shape}, y shape: {y.shape}")

# Train-validation-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42, stratify=y_train)

# Reshape for CNN (add channel dimension)
X_train = X_train[..., np.newaxis]
X_val = X_val[..., np.newaxis]
X_test = X_test[..., np.newaxis]
input_shape = X_train.shape[1:4]
print(f"X_train shape: {X_train.shape}, X_val shape: {X_val.shape}, X_test shape: {X_test.shape}")

# Define Short-chunk CNN model
model_short_cnn = Sequential([
    Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape),
    BatchNormalization(),
    MaxPooling2D((2, 2), padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2), padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2), padding='same'),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2), padding='same'),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2), padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2), padding='same'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2), padding='same'),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

# Compile model
model_short_cnn.compile(
    optimizer=Adam(learning_rate=0.0115),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Train model
model_short_cnn.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=32,
    epochs=100,
    callbacks=[early_stopping],
    verbose=1
)

# Evaluate on test set
loss, accuracy = model_short_cnn.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

Original MFCCs length: 9986, Labels length: 9986
Aligned MFCCs length: 9986, Labels length: 9986
X shape: (9986, 130, 13), y shape: (9986,)
X_train shape: (4893, 130, 13, 1), X_val shape: (2097, 130, 13, 1), X_test shape: (2996, 130, 13, 1)
Epoch 1/100


c:\Users\botla\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


153/153 ━━━━━━━━━━━━━━━━━━━━ 22s 118ms/step - accuracy: 0.2367 - loss: 2.2805 - val_accuracy: 0.1907 - val_loss: 4.5454
Epoch 2/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 18s 115ms/step - accuracy: 0.3134 - loss: 1.8722 - val_accuracy: 0.3433 - val_loss: 2.2137
Epoch 3/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 18s 118ms/step - accuracy: 0.3929 - loss: 1.6958 - val_accuracy: 0.3901 - val_loss: 1.5150
Epoch 4/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 18s 115ms/step - accuracy: 0.4225 - loss: 1.5930 - val_accuracy: 0.3968 - val_loss: 1.9594
Epoch 5/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 17s 113ms/step - accuracy: 0.4610 - loss: 1.4787 - val_accuracy: 0.3972 - val_loss: 1.7774
Epoch 6/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 17s 113ms/step - accuracy: 0.4859 - loss: 1.4368 - val_accuracy: 0.4130 - val_loss: 1.9370
Epoch 7/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 18s 116ms/step - accuracy: 0.5126 - loss: 1.3268 - val_accuracy: 0.4034 - val_loss: 2.3632
Epoch 8/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 18s 119ms/step - accuracy: 0.5424 - loss: 1.308

In [1]:
# Save model
model_short_cnn.save('../models/short_chunk_cnn.keras')

NameError: name 'model_short_cnn' is not defined